In [46]:
#from twitter_preprocess import *
import csv
import argparse
import pandas as pd
import re
import os
from collections import Counter

TRAIN = '../data/train/training_data.csv'

# get unigram counts for data
def get_unigrams():
    train_data = pd.read_csv(TRAIN, index_col=0)
    unigrams = Counter()
    tweets = train_data[['tweet']]
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        for word in s:
            unigrams[word] += 1
    return unigrams
    
def get_bigrams():
    train_data = pd.read_csv(TRAIN, index_col=0)
    bigrams = Counter()
    tweets = train_data[['tweet']]
    start = "<s>"
    end = "</s>"
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        word = start + " " + s[0]
        bigrams[word] += 1
        for i in range(len(s)) - 1:
            word = s[i] + " " + s[i+1]
            bigrams[word] += 1
        word = s[len(s) - 1] + " " + end
        bigrams[word] += 1
    print(bigrams)
    return bigrams

In [47]:
get_bigrams()

TypeError: unsupported operand type(s) for -: 'range' and 'int'